In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [65]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibmq_toronto')
device

<IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

32000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69710/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [16]:
#checking circuit

In [17]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [18]:
circuit = front_layer.copy()

In [19]:
apply_checking_circuit(circuit, [0], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [0], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69710/1937118134.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69710/1937118134.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [20]:
circuit.draw()

┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [21]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [22]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [23]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69710/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [24]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐               
«q_0: ──┤ U(π/2,0,π) ├───────────────
«       ├────────────┤               
«q_1: ──┤ U(π/2,0,π) ├───────────────
«     ┌─┴────────────┴┐┌────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├
«     └┬──────────────┤├────────────┤
«q_3: ─┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«      └──────────────┘└────────────┘
«q_4: ───────────────────────────────
«

In [25]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [26]:
cuts = [(circuit.qubits[0], 3),(circuit.qubits[0], 4)]

In [27]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«      ┌──────────────┐                
«q_0: ─┤ U(π/2,0,π/2) ├────────────────
«      ├──────────────┤                
«q_1: ─┤ U(π/2,0,π/2) ├────────────────
«     ┌┴──────────────┤┌──────────────┐
«q_2: ┤ Ry(0.003620

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐               
«q_0: ──┤ U(π/2,0,π) ├───────────────
«       └────────────┘               
«q_1: ───────────────────────────────
«     ┌───────────────┐              
«q_2: ┤ Ry(0.0036205) ├──────

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐               
«q_0: ──┤ U(π/2,0,π) ├───────────────
«       ├────────────┤               
«q_1: ──┤ U(π/2,0,π) ├───────────────
«     ┌─┴────────────┴┐┌────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/

In [28]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 2)), (1, Qubit(QuantumRegister(4, 'q'), 0)), (0, Qubit(QuantumRegister(3, 'q'), 0))), Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 1)),)}


In [29]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [30]:
total_variants

24

In [31]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [32]:
fragments_list[0][0].draw()

┌────────────────┐┌──────────────┐
q_0: ──────────────────────────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├
     ┌───┐                     │ └─────┬───┬──────┘└──────────────┘
q_1: ┤ H ├──────────────────■──■───────┤ H ├───────────────────────
     ├───┤┌───────────────┐ │          └───┘                       
q_2: ┤ X ├┤ Ry(-0.003534) ├─■──────────────────────────────────────
     └───┘└───────────────┘

In [33]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
                                 ┌────────────────┐┌──────────────┐
q_0: ──────────────────────────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├
     ┌───┐                     │ └─────┬───┬──────┘└──────────────┘
q_1: ┤ H ├──────────────────■──■───────┤ H ├───────────────────────
     ├───┤┌───────────────┐ │          └───┘                       
q_2: ┤ X ├┤ Ry(-0.003534) ├─■──────────────────────────────────────
     └───┘└───────────────┘                                        

fragment 1:
                                                                               »
q_0: ───────────────────────────────────■──────────────────────────────────────»
           ┌───┐      ┌───────────────┐ │    ┌───────────────┐ ┌──────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─┤ Ry(-0.015751) ├─┤ U(π/2,0,π/2) ├»
     ┌─────┴───┴─────┐└───────────────┘    │ └───────────────┘ ├──────────────┤»
q_2: ┤ Ry(-0.019851) ├─────────────────────┼─────────■─────────┤ Ry(0.019873) ├»
     ├───────

In [34]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [35]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 2)),
  (1, Qubit(QuantumRegister(4, 'q'), 0)),
  (0, Qubit(QuantumRegister(3, 'q'), 0))),
 Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 1)),)}

In [36]:
num_fragments = len(fragments)
num_fragments

2

In [37]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [38]:
fragments[0].draw()

┌────────────────┐
q_0: ──────────────────────────■─┤ Ry(-0.0036834) ├
     ┌───┐                     │ └─────┬───┬──────┘
q_1: ┤ H ├──────────────────■──■───────┤ H ├───────
     ├───┤┌───────────────┐ │          └───┘       
q_2: ┤ X ├┤ Ry(-0.003534) ├─■──────────────────────
     └───┘└───────────────┘

In [39]:
fragments[1].draw()

q_0: ───────────────────────────────────■──────────────────────────────────────
           ┌───┐      ┌───────────────┐ │    ┌───────────────┐                 
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─┤ Ry(-0.015751) ├─────────────────
     ┌─────┴───┴─────┐└───────────────┘    │ └───────────────┘ ┌──────────────┐
q_2: ┤ Ry(-0.019851) ├─────────────────────┼─────────■─────────┤ Ry(0.019873) ├
     ├───────────────┤                     │         │        ┌┴──────────────┤
q_3: ┤ Ry(0.0038243) ├─────────────────────■─────────■────────┤ Ry(0.0036205) ├
     └───────────────┘                                        └───────────────┘

In [40]:
hardware_index = 1

In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, initial_layout = [23,25,24,26], opt_lvl = 3, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 25 basis
len(tomography_circuits)

25

In [43]:
#contains three fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [47]:
multi_exp_circuits[1][-200].draw()

┌─────────────────┐ ░                                         »
  q_0: ┤ U3(-1.9106,π,0) ├─░────────────────────────────────────■────»
       └─────────────────┘ ░       ┌───┐      ┌───────────────┐ │    »
  q_1: ────────────────────░───────┤ X ├──────┤ Ry(-0.015869) ├─■──■─»
                           ░ ┌─────┴───┴─────┐└───────────────┘    │ »
  q_2: ────────────────────░─┤ Ry(-0.019851) ├─────────────────────┼─»
                           ░ ├───────────────┤                     │ »
  q_3: ────────────────────░─┤ Ry(0.0038243) ├─────────────────────■─»
                           ░ └───────────────┘                       »
c17_0: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_1: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_2: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_3: ══════════════════════════════════════════════════════════════»
                                                                     »
«                                                        ░ ┌───┐      ┌─┐
«  q_0: ─────────────────────────────────────────────────░─┤ H ├──────┤M├
«       ┌───────────────┐ ┌──────────────┐               ░ └┬─┬┘      └╥┘
«  q_1: ┤ Ry(-0.015751) ├─┤ U(π/2,0,π/2) ├───────────────░──┤M├────────╫─
«       └───────────────┘ ├──────────────┤┌────────────┐ ░  └╥┘ ┌─┐    ║ 
«  q_2: ────────■─────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├─░───╫──┤M├────╫─
«               │        ┌┴──────────────┤├────────────┤ ░   ║  └╥┘┌─┐ ║ 
«  q_3: ────────■────────┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├─░───╫───╫─┤M├─╫─
«                        └───────────────┘└────────────┘ ░   ║   ║ └╥┘ ║ 
«c17_0: ═════════════════════════════════════════════════════╬═══╬══╬══╩═
«                                                            ║   ║  ║    
«c17_1: ═════════════════════════════════════════════════════╩═══╬══╬════
«                                                                ║  ║    
«c17_2: ═════════════════════════════════════════════════════════╩══╬════
«                                                                   ║    
«c17_3: ════════════════════════════════════════════════════════════╩════
«

In [49]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [16,14,8,11], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  6647acac-3aba-48e1-a697-fc2fd684e060
Job Status: job has successfully run
job_id:  63777e8104e46ab4b8625611
Job Status: job is queued (13)    

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/63777e8104e46ab4b8625611/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7f84da54c5e0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [83]:
frag_datas[1] = [device.retrieve_job(job_id='63777e8104e46ab4b8625611').result()]

In [90]:
frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
                             backend = simulation_backend, shots = shots // total_variants,
                             optimization_level = 3, monitor_jobs = True, noise_model = None)

job_id:  fe1b1099-b285-436b-9530-0b792a74a9d9
Job Status: job has successfully run


In [91]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z'))"

In [92]:
num_tomo_circuits

25

In [93]:
test = [[], []]

In [94]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [95]:
frag_datas[1][0].get_counts()

[{'0000': 3663,
  '0001': 64,
  '0010': 3917,
  '0011': 51,
  '0100': 4093,
  '0101': 50,
  '0110': 4134,
  '0111': 43,
  '1000': 3802,
  '1001': 51,
  '1010': 3580,
  '1011': 57,
  '1100': 4135,
  '1101': 59,
  '1110': 4240,
  '1111': 61},
 {'0000': 1897,
  '0001': 1705,
  '0010': 2068,
  '0011': 1868,
  '0100': 2210,
  '0101': 2158,
  '0110': 2101,
  '0111': 2029,
  '1000': 1943,
  '1001': 1850,
  '1010': 1782,
  '1011': 1748,
  '1100': 2247,
  '1101': 2074,
  '1110': 2200,
  '1111': 2120},
 {'0000': 1723,
  '0001': 1971,
  '0010': 1926,
  '0011': 2167,
  '0100': 1953,
  '0101': 2325,
  '0110': 1997,
  '0111': 2153,
  '1000': 1790,
  '1001': 2042,
  '1010': 1680,
  '1011': 1771,
  '1100': 1915,
  '1101': 2290,
  '1110': 1962,
  '1111': 2335},
 {'0000': 1204,
  '0001': 2463,
  '0010': 1435,
  '0011': 2428,
  '0100': 1428,
  '0101': 2680,
  '0110': 1520,
  '0111': 2708,
  '1000': 1246,
  '1001': 2296,
  '1010': 1337,
  '1011': 2431,
  '1100': 1521,
  '1101': 2722,
  '1110': 1636,
  '11

In [96]:
frag_datas[0][0].get_counts()

[{'111': 15915, '110': 16085},
 {'111': 7998, '011': 8101, '110': 7962, '010': 7939},
 {'001': 1, '111': 8078, '011': 7951, '110': 7995, '010': 7975},
 {'100': 10828, '111': 5344, '101': 10594, '110': 5234},
 {'110': 2664,
  '100': 5316,
  '001': 5313,
  '101': 5336,
  '000': 5407,
  '010': 2661,
  '111': 2694,
  '011': 2609},
 {'011': 2609,
  '111': 2737,
  '010': 2652,
  '000': 5339,
  '001': 5410,
  '100': 5311,
  '110': 2614,
  '101': 5328},
 {'100': 10627, '111': 5167, '110': 5278, '101': 10928},
 {'011': 2692,
  '111': 2683,
  '101': 5358,
  '110': 2649,
  '100': 5284,
  '001': 5301,
  '000': 5382,
  '010': 2651},
 {'110': 2714,
  '001': 5307,
  '100': 5394,
  '101': 5384,
  '111': 2690,
  '011': 2625,
  '010': 2533,
  '000': 5353},
 {'111': 5348, '110': 5359, '101': 10588, '100': 10705},
 {'011': 2636,
  '111': 2599,
  '100': 5467,
  '110': 2582,
  '001': 5327,
  '101': 5342,
  '000': 5336,
  '010': 2711},
 {'110': 2683,
  '100': 5418,
  '001': 5336,
  '011': 2718,
  '111': 2590

In [97]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S0', 'S0', 'S0') ('Z', 'Z', 'Z')
(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S0',) ('Z',) [0] [2]
(('S0', 'S0', 'S0'), ('Z', 'Z', 'X')) ('S0', 'S0', 'S0') ('Z', 'Z', 'X')
(('S0', 'S0', 'S0'), ('Z', 'Z', 'X')) ('S0',) ('X',) [0] [2]
(('S0', 'S0', 'S0'), ('Z', 'Z', 'Y')) ('S0', 'S0', 'S0') ('Z', 'Z', 'Y')
(('S0', 'S0', 'S0'), ('Z', 'Z', 'Y')) ('S0',) ('Y',) [0] [2]
(('S1', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S1', 'S0', 'S0') ('Z', 'Z', 'Z')
(('S1', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S1',) ('Z',) [0] [2]
(('S1', 'S0', 'S0'), ('Z', 'Z', 'X')) ('S1', 'S0', 'S0') ('Z', 'Z', 'X')
(('S1', 'S0', 'S0'), ('Z', 'Z', 'X')) ('S1',) ('X',) [0] [2]
(('S1', 'S0', 'S0'), ('Z', 'Z', 'Y')) ('S1', 'S0', 'S0') ('Z', 'Z', 'Y')
(('S1', 'S0', 'S0'), ('Z', 'Z', 'Y')) ('S1',) ('Y',) [0] [2]
(('S2', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S2', 'S0', 'S0') ('Z', 'Z', 'Z')
(('S2', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S2',) ('Z',) [0] [2]
(('S2', 'S0', 'S0'), ('Z', 'Z', 'X')) ('S2', 'S0', 'S0') ('Z',

In [98]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1


In [99]:
len(organized_data)

25

In [100]:
organized_data[2]

[{'11': {(('S0',), ('Zm',)): 1,
   (('S3',), ('Xm',)): 1,
   (('S0',), ('Zp',)): 0,
   (('S0',), ('Xp',)): 0,
   (('S0',), ('Xm',)): 0,
   (('S0',), ('Yp',)): 0,
   (('S0',), ('Ym',)): 0,
   (('S1',), ('Zp',)): 0,
   (('S1',), ('Zm',)): 0,
   (('S1',), ('Xp',)): 0,
   (('S1',), ('Xm',)): 0,
   (('S1',), ('Yp',)): 0,
   (('S1',), ('Ym',)): 0,
   (('S2',), ('Zp',)): 0,
   (('S2',), ('Zm',)): 0,
   (('S2',), ('Xp',)): 0,
   (('S2',), ('Xm',)): 0,
   (('S2',), ('Yp',)): 0,
   (('S2',), ('Ym',)): 0,
   (('S3',), ('Zp',)): 0,
   (('S3',), ('Zm',)): 0,
   (('S3',), ('Xp',)): 0,
   (('S3',), ('Yp',)): 0,
   (('S3',), ('Ym',)): 0},
  '10': {(('S0',), ('Zm',)): 31999,
   (('S0',), ('Xm',)): 16230,
   (('S0',), ('Xp',)): 15770,
   (('S0',), ('Yp',)): 15925,
   (('S0',), ('Ym',)): 16075,
   (('S1',), ('Zm',)): 10522,
   (('S1',), ('Xp',)): 5206,
   (('S1',), ('Xm',)): 5352,
   (('S1',), ('Ym',)): 5274,
   (('S1',), ('Yp',)): 5314,
   (('S2',), ('Zm',)): 10654,
   (('S2',), ('Xm',)): 5298,
   (('S2

In [101]:
organized_data[-1][-1]

{'000': {(('S0',), ('Zp',)): 1011,
  (('S0',), ('Zm',)): 28,
  (('S0',), ('Xp',)): 492,
  (('S0',), ('Xm',)): 526,
  (('S0',), ('Yp',)): 490,
  (('S0',), ('Ym',)): 512,
  (('S1',), ('Zp',)): 326,
  (('S1',), ('Zm',)): 620,
  (('S1',), ('Xp',)): 169,
  (('S1',), ('Xm',)): 783,
  (('S1',), ('Yp',)): 543,
  (('S1',), ('Ym',)): 434,
  (('S2',), ('Zp',)): 325,
  (('S2',), ('Zm',)): 650,
  (('S2',), ('Xp',)): 654,
  (('S2',), ('Xm',)): 270,
  (('S2',), ('Yp',)): 776,
  (('S2',), ('Ym',)): 193,
  (('S3',), ('Zp',)): 360,
  (('S3',), ('Zm',)): 583,
  (('S3',), ('Xp',)): 594,
  (('S3',), ('Xm',)): 332,
  (('S3',), ('Yp',)): 187,
  (('S3',), ('Ym',)): 734},
 '001': {(('S0',), ('Zp',)): 29789,
  (('S0',), ('Zm',)): 464,
  (('S0',), ('Xp',)): 15889,
  (('S0',), ('Xm',)): 14490,
  (('S0',), ('Yp',)): 14264,
  (('S0',), ('Ym',)): 16144,
  (('S1',), ('Zp',)): 10478,
  (('S1',), ('Zm',)): 19943,
  (('S1',), ('Xp',)): 1872,
  (('S1',), ('Xm',)): 28568,
  (('S1',), ('Yp',)): 15134,
  (('S1',), ('Ym',)):

In [102]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'11': {(('S0',), ('Zm',)): 15915, (('S0',), ('Xm',)): 7998, (('S0',), ('Xp',)): 8101, (('S0',), ('Ym',)): 8078, (('S0',), ('Yp',)): 7951, (('S1',), ('Zm',)): 5344, (('S1',), ('Xm',)): 2694, (('S1',), ('Xp',)): 2609, (('S1',), ('Yp',)): 2609, (('S1',), ('Ym',)): 2737, (('S2',), ('Zm',)): 5167, (('S2',), ('Xp',)): 2692, (('S2',), ('Xm',)): 2683, (('S2',), ('Ym',)): 2690, (('S2',), ('Yp',)): 2625, (('S3',), ('Zm',)): 5348, (('S3',), ('Xp',)): 2636, (('S3',), ('Xm',)): 2599, (('S3',), ('Yp',)): 2718, (('S3',), ('Ym',)): 2590, (('S0',), ('Zp',)): 0, (('S1',), ('Zp',)): 0, (('S2',), ('Zp',)): 0, (('S3',), ('Zp',)): 0}, '10': {(('S0',), ('Zm',)): 16085, (('S0',), ('Xm',)): 7962, (('S0',), ('Xp',)): 7939, (('S0',), ('Ym',)): 7995, (('S0',), ('Yp',)): 7975, (('S1',), ('Zm',)): 5234, (('S1',), ('Xm',)): 2664, (('S1',), ('Xp',)): 2661, (('S1',), ('Yp',)): 2652, (('S1',), ('Ym',)): 2614, (('S2',), ('Zm',)): 5278, (('S2',), ('Xm',)): 2649, (('S2',), ('Xp',)): 2651, (('S2',), ('Ym',)): 2714, (('S2'

{'10': {(('S0',), ('Zm',)): 16087, (('S0',), ('Xp',)): 7977, (('S0',), ('Xm',)): 8031, (('S0',), ('Ym',)): 7959, (('S0',), ('Yp',)): 8054, (('S1',), ('Zm',)): 5242, (('S1',), ('Xm',)): 2675, (('S1',), ('Xp',)): 2646, (('S1',), ('Ym',)): 2704, (('S1',), ('Yp',)): 2669, (('S2',), ('Zm',)): 5319, (('S2',), ('Xm',)): 2686, (('S2',), ('Xp',)): 2687, (('S2',), ('Yp',)): 2652, (('S2',), ('Ym',)): 2624, (('S3',), ('Zm',)): 5274, (('S3',), ('Xm',)): 2636, (('S3',), ('Xp',)): 2689, (('S3',), ('Yp',)): 2684, (('S3',), ('Ym',)): 2619, (('S0',), ('Zp',)): 0, (('S1',), ('Zp',)): 0, (('S2',), ('Zp',)): 0, (('S3',), ('Zp',)): 0}, '11': {(('S0',), ('Zm',)): 15913, (('S0',), ('Xp',)): 8031, (('S0',), ('Xm',)): 7961, (('S0',), ('Ym',)): 8061, (('S0',), ('Yp',)): 7926, (('S1',), ('Zm',)): 5428, (('S1',), ('Xp',)): 2736, (('S1',), ('Xm',)): 2586, (('S1',), ('Yp',)): 2631, (('S1',), ('Ym',)): 2736, (('S2',), ('Zm',)): 5372, (('S2',), ('Xm',)): 2638, (('S2',), ('Xp',)): 2672, (('S2',), ('Yp',)): 2713, (('S2'

{'000': {(('S0',), ('Zp',)): 9489, (('S0',), ('Zm',)): 151, (('S0',), ('Xp',)): 5017, (('S0',), ('Xm',)): 4821, (('S0',), ('Yp',)): 4591, (('S0',), ('Ym',)): 5307, (('S1',), ('Zp',)): 2818, (('S1',), ('Zm',)): 5314, (('S1',), ('Xp',)): 514, (('S1',), ('Xm',)): 7622, (('S1',), ('Yp',)): 4179, (('S1',), ('Ym',)): 3985, (('S2',), ('Zp',)): 2777, (('S2',), ('Zm',)): 5360, (('S2',), ('Xp',)): 5883, (('S2',), ('Xm',)): 2280, (('S2',), ('Yp',)): 7033, (('S2',), ('Ym',)): 933, (('S3',), ('Zp',)): 2855, (('S3',), ('Zm',)): 5113, (('S3',), ('Xp',)): 5752, (('S3',), ('Xm',)): 2235, (('S3',), ('Yp',)): 989, (('S3',), ('Ym',)): 7178}, '001': {(('S0',), ('Zp',)): 7210, (('S0',), ('Zm',)): 112, (('S0',), ('Xp',)): 3962, (('S0',), ('Xm',)): 3401, (('S0',), ('Yp',)): 3472, (('S0',), ('Ym',)): 3764, (('S1',), ('Zp',)): 2858, (('S1',), ('Zm',)): 5333, (('S1',), ('Xp',)): 522, (('S1',), ('Xm',)): 7671, (('S1',), ('Yp',)): 4006, (('S1',), ('Ym',)): 4282, (('S2',), ('Zp',)): 2980, (('S2',), ('Zm',)): 5293, 

{'000': {(('S0',), ('Zp',)): 4912, (('S0',), ('Zm',)): 87, (('S0',), ('Xp',)): 2632, (('S0',), ('Xm',)): 2461, (('S0',), ('Yp',)): 2303, (('S0',), ('Ym',)): 2744, (('S1',), ('Zp',)): 1495, (('S1',), ('Zm',)): 2759, (('S1',), ('Xp',)): 281, (('S1',), ('Xm',)): 4062, (('S1',), ('Yp',)): 2277, (('S1',), ('Ym',)): 2099, (('S2',), ('Zp',)): 1525, (('S2',), ('Zm',)): 2890, (('S2',), ('Xp',)): 3176, (('S2',), ('Xm',)): 1193, (('S2',), ('Yp',)): 3851, (('S2',), ('Ym',)): 484, (('S3',), ('Zp',)): 1608, (('S3',), ('Zm',)): 2769, (('S3',), ('Xp',)): 3097, (('S3',), ('Xm',)): 1216, (('S3',), ('Yp',)): 492, (('S3',), ('Ym',)): 3790}, '001': {(('S0',), ('Zp',)): 3779, (('S0',), ('Zm',)): 47, (('S0',), ('Xp',)): 2087, (('S0',), ('Xm',)): 1817, (('S0',), ('Yp',)): 1790, (('S0',), ('Ym',)): 2090, (('S1',), ('Zp',)): 1483, (('S1',), ('Zm',)): 2893, (('S1',), ('Xp',)): 241, (('S1',), ('Xm',)): 3964, (('S1',), ('Yp',)): 2032, (('S1',), ('Ym',)): 2221, (('S2',), ('Zp',)): 1435, (('S2',), ('Zm',)): 2672, ((

In [103]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [104]:
direct_recombined_dist

{'10000': 5.278240360585601e-06,
 '10010': 0.03253173995987003,
 '11000': -4.1946500271512693e-05,
 '11010': -0.00045328337321746005,
 '10100': 2.6237868042325953e-05,
 '10110': -0.00031754180067900976,
 '11110': -3.164839273632337e-05,
 '11100': 5.033047393372716e-06,
 '00000': -1.0595499819771535e-07,
 '00010': -9.042151992549381e-06,
 '01000': -3.878498463695283e-09,
 '01010': -1.1213675049351706e-07,
 '00100': 8.667311793516713e-10,
 '00110': -5.080518923501507e-08,
 '01110': 8.53721250732707e-10,
 '01100': 1.0852751443606086e-09,
 '00001': 0.02847838531234062,
 '00011': 0.9204880578812592,
 '01001': 0.00042574516947618184,
 '01011': 0.008858881382641548,
 '00101': 0.00032228388772400577,
 '00111': 0.009582060307335407,
 '01111': 0.00012483204224858702,
 '01101': 5.1970899140498075e-06}

In [105]:
likely_recombined_dist

{'10000': 0.00025076574996490877,
 '10010': 0.032663364500163994,
 '11010': 1.6871583899697363e-05,
 '10110': 1.3762140112192701e-05,
 '00001': 0.02717766840845698,
 '00011': 0.9236870068002837,
 '01011': 0.007801437762699893,
 '00111': 0.008389123054418687}

In [106]:
filtered_recombined_lists

[{'0001': 0.056774542708160486,
  '0011': 0.05751922182817814,
  '1001': 0.060538927937173734,
  '1011': 0.06439226086882677,
  '0101': 0.052575408062851506,
  '0111': 0.0568215854278467,
  '1101': 0.06260805303769718,
  '1111': 0.06798729803730688,
  '0000': 0.05713805363166819,
  '0010': 0.05791853044280965,
  '1000': 0.06092791098223776,
  '1010': 0.06484177776236677,
  '0100': 0.05292400735861365,
  '0110': 0.057206165903707806,
  '1100': 0.06301023299614138,
  '1110': 0.06848173686428398},
 {'0000': 0.21943253722526426,
  '0010': 0.2531369759242419,
  '1000': 0.0017562644933102035,
  '1010': 0.0020882004160522657,
  '0100': 0.001668938795759644,
  '0110': 0.0017846572814791948,
  '0001': 0.217887733058919,
  '0011': 0.251446582815086,
  '1001': 0.0017456238020045576,
  '1011': 0.002074134482458858,
  '0101': 0.001655915882902319,
  '0111': 0.001773447139994877},
 {'0001': 0.007185819496922788,
  '0011': 0.23252482410225944,
  '1001': 0.008385569901408943,
  '1011': 0.2587887735292

In [107]:
from utils.utils import filter_results

In [108]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [109]:
filter_direct_recombined

{'0000': -1.0595499819771535e-07,
 '0010': -9.042151992549381e-06,
 '1000': -3.878498463695283e-09,
 '1010': -1.1213675049351706e-07,
 '0100': 8.667311793516713e-10,
 '0110': -5.080518923501507e-08,
 '1110': 8.53721250732707e-10,
 '1100': 1.0852751443606086e-09,
 '0001': 0.02847838531234062,
 '0011': 0.9204880578812592,
 '1001': 0.00042574516947618184,
 '1011': 0.008858881382641548,
 '0101': 0.00032228388772400577,
 '0111': 0.009582060307335407,
 '1111': 0.00012483204224858702,
 '1101': 5.1970899140498075e-06}

In [110]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [111]:
filter_likely_recombined

{'0001': 0.02717766840845698,
 '0011': 0.9236870068002837,
 '1011': 0.007801437762699893,
 '0111': 0.008389123054418687}

In [112]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [113]:
likely_recombined_dist['00011']

0.9236870068002837

In [114]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

0.9551543411276088

In [115]:
norm_filter_dist

{'0001': 0.028103532658738682,
 '0011': 0.9551543411276088,
 '1011': 0.00806721009521661,
 '0111': 0.008674916118436031}

In [116]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [117]:
final_expect_val

(-1.0345563346769033+0j)

In [118]:
import json
with open('LiH_toronto_check0.json', 'w') as f:
    json.dump(norm_filtered_lists, f)